# Multiple Linear Regression

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.linear_model import LinearRegression

## Load the Data

In [22]:
data = pd.read_csv('data.csv')
data.head()

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83


In [23]:
data.describe()

,SAT,"Rand 1,2,3",GPA
count,84.000000,84.000000,84.000000
mean,1845.273810,2.059524,3.330238
std,104.530661,0.855192,0.271617
min,1634.000000,1.000000,2.400000
25%,1772.000000,1.000000,3.190000
50%,1846.000000,2.000000,3.380000
75%,1934.000000,3.000000,3.502500
max,2050.000000,3.000000,3.810000


### Declaring Variables

In [24]:
x = data[['SAT', 'Rand 1,2,3']]
y = data['GPA']

### Regression

In [25]:
reg = LinearRegression()
reg.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [26]:
reg.coef_

array([ 0.00165354, -0.00826982])

In [27]:
reg.intercept_

0.29603261264909486

### Calculating the R-squared

In [28]:
reg.score(x,y)

0.4066811952814283

### How do we get Adjusted R-Squared?

$R^2_{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$

In [29]:
x.shape

(84, 2)

In [30]:
r2 = reg.score(x,y)

n = x.shape[0] #n=84 (the number of observation)
p = x.shape[1] #p=2 (the number of predictors)

adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.3920313482513401

### Feature Selection, how do we get p-values with SKLearn?!

In [31]:
from sklearn.feature_selection import f_regression

In [32]:
f_regression(x,y)

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [33]:
#Second array is our p value
p_values = f_regression(x,y)[1]
p_values

array([7.19951844e-11, 6.76291372e-01])

In [34]:
p_values.round(3)

array([0.   , 0.676])

#### As we can see. P-value of rand1,2,3 is 0.676, which means this variable is useless to get GPA.
# Let's summary table like Statsmodels summary table

In [35]:
reg_summary = pd.DataFrame(data = x.columns.values, columns=['Features'])
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [36]:
reg_summary ['Coefficients'] = reg.coef_
reg_summary ['p-values'] = p_values.round(3)

In [37]:
reg_summary

,Features,Coefficients,p-values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


# ---------------------------------------------------------

## Standartization / Feature scaling / Normalization

In [38]:
from sklearn.preprocessing import StandardScaler

In [39]:
scaler = StandardScaler()

In [40]:
scaler.fit(x)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [41]:
x_scaled = scaler.transform(x)

In [44]:
x_scaled

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974],
       [-1.54247971,  1.10632974],
       [-1.46548748, -0.07002087],
       [-1.68684014, -1.24637147],
       [-0.78218146, -0.07002087],
       [-0.78218146, -1.24637147],
       [-0.51270866, -0.07002087],
       [ 0.04548499,  1.10632974],
       [-1.06127829,  1.10632974],
       [-0.67631715, -0.07002087],
       [-1.06127829, -1.24637147],
       [-1.28263094,  1.10632974],
       [-0.6955652 , -0.07002087],
       [ 0.25721362, -0.07002087],
       [-0.86879772,  1.10632974],
       [-1.64834403, -0.07002087],
       [-0.03150724,  1.10632974],
       [-0.57045283,  1.10632974],
       [-0.81105355,  1.10632974],
       [-1.18639066,  1.10632974],
       [-1.75420834,  1.10632974],
       [-1.52323165, -1.24637147],
       [ 1.23886453, -1.24637147],
       [-0.18549169, -1.24637147],
       [-0.5608288 , -1.24637147],
       [-0.23361183,  1.10632974],
       [ 1.68156984,

## Regression with scaled features

In [45]:
reg = LinearRegression()

In [46]:
reg.fit(x_scaled, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [48]:
reg.coef_

array([ 0.17181389, -0.00703007])

In [49]:
reg.intercept_

3.330238095238095

## Creating summary table

In [53]:
reg_summary = pd.DataFrame([['Intercept'], ['SAT'], ['Rand 1,2,3']], columns=['Features'])
reg_summary['Weights'] = reg.intercept_, reg.coef_[0], reg.coef_[1]

In [54]:
reg_summary

,Features,Weights
0,Intercept,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


## Making predictions with the standarized coefficents (Weights)

In [55]:
new_data = pd.DataFrame(data=[[1700, 2],[1800,1]], columns=['SAT', 'Rand 1,2,3'])

In [56]:
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1800,1


In [57]:
reg.predict(new_data)

array([295.39979563, 312.58821497])

In [59]:
new_data_scaled = scaler.transform(new_data)
new_data_scaled

array([[-1.39811928, -0.07002087],
       [-0.43571643, -1.24637147]])

In [60]:
reg.predict(new_data_scaled)

array([3.09051403, 3.26413803])

## What if we delete the 'Rand 1,2,3' variable?

In [62]:
reg_simple = LinearRegression()
x_simple_matrix = x_scaled[:,0].reshape(-1,1)
reg_simple.fit(x_simple_matrix, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [64]:
reg_simple.predict(new_data_scaled[:,0].reshape(-1,1))

array([3.08970998, 3.25527879])

## As we can see predicted Their weights will be so close to 